In [1]:
import os
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from dotenv import load_dotenv

In [2]:
# Carregar variáveis de ambiente
load_dotenv()

# Obter variáveis de ambiente
user = os.environ.get('DB_USER')
password = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOST')
port = os.environ.get('DB_PORT', '5432')
database = os.environ.get('DB_NAME')

# Criar string de conexão
connection_string = f'postgresql://{user}:{password}@{host}:{port}/{database}'

# Criar conexão com o banco de dados
engine = create_engine(connection_string)

In [3]:
def load_data_in_chunks(query, engine, chunksize=50000):
    # Lista para armazenar os DataFrames Dask
    df_list = []
    
    # Ler os dados em chunks usando pandas
    for chunk in pd.read_sql(query, engine, chunksize=chunksize):
        # Converter o chunk pandas DataFrame para Dask DataFrame
        ddf_chunk = dd.from_pandas(chunk, npartitions=1)
        df_list.append(ddf_chunk)
    
    # Concatenar todos os Dask DataFrames em um único DataFrame Dask
    df = dd.concat(df_list)
    return df

In [4]:
query = 'SELECT * FROM ds_market'
df = load_data_in_chunks(query, engine)

In [5]:
# Agrupar por 'item' e somar as vendas
item_sales = df.groupby('item')['sales'].sum().compute()

# Converter para um DataFrame pandas para facilitar a manipulação
item_sales = item_sales.reset_index()